In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow.keras import layers
import time
from tensorflow import data
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense,ZeroPadding2D, BatchNormalization, Activation, Layer, ReLU, LeakyReLU,Conv2D,AveragePooling2D,UpSampling2D,Reshape,Flatten
from tensorflow.keras.models import Model
from keras import backend


In [2]:
BATCH_SIZE = 962


In [3]:
data = np.load('finDat.npy')

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(60000).batch(962)


Metal device set to: Apple M2 Max


2023-02-19 23:19:51.135499: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-19 23:19:51.135517: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
DSHAPE = 151

In [6]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, activation='relu', use_bias=False, input_shape=(DSHAPE,)))
    model.add(layers.BatchNormalization())

    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(128,activation='relu'))


    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))

    model.add(layers.Dense(DSHAPE, activation='relu', use_bias=False, input_shape=(DSHAPE,)))


    

    return model

In [7]:
generator = make_generator_model()

noise = tf.random.normal([1, DSHAPE])
generated_image = generator(noise, training=False)


In [8]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, activation='relu', use_bias=False, input_shape=(DSHAPE,)))
    model.add(layers.BatchNormalization())

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.3))
   
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='relu'))
    return model

In [9]:
discriminator = make_discriminator_model()

In [12]:
def discriminator_loss(real_output, fake_output):
    return tf.reduce_mean(real_output*(1-fake_output))

In [13]:
def generator_loss(fake_output):
    return tf.math.reduce_mean((1-fake_output)**2)

In [14]:
generator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)

In [15]:
checkpoint_dir = './training_checkpointsL2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [16]:
EPOCHS = 500
noise_dim = DSHAPE
num_examples_to_generate = 50

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [17]:
train_dataset

<BatchDataset element_spec=TensorSpec(shape=(None, 151), dtype=tf.float32, name=None)>

In [18]:
def train_step(images, gen_losses, disc_losses):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)

        disc_loss = discriminator_loss(real_output, fake_output)
        gen_losses = gen_losses.append(gen_loss.numpy())
        disc_losses = disc_losses.append(disc_loss.numpy())
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [19]:

def train(dataset, epochs, gen_losses, disc_losses, gloss, dloss):
  for epoch in range(epochs):
    start = time.time()


    for image_batch in dataset:
        train_step(image_batch, gen_losses,disc_losses)
    print("gen_loss =" + str(gen_losses[-1]))
    print("disc_loss =" + str(disc_losses[-1]))
    gloss.append(gen_losses[-1])
    dloss.append(disc_losses[-1])

    # Produce images for the GIF as you go
    x = generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  saved = generate_and_save_images(generator,
                           epochs,
                           seed)
  return saved

In [20]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)
  return predictions


In [21]:
gen_losses = []
disc_losses = []
gloss = []
dloss = []

In [22]:
final = train(train_dataset,EPOCHS, gen_losses, disc_losses, gloss, dloss)


gen_loss =0.91531485
disc_loss =0.013139045
Time for epoch 1 is 2.31656813621521 sec
gen_loss =0.9150947
disc_loss =0.0032595934
Time for epoch 2 is 0.27500200271606445 sec
gen_loss =0.91589767
disc_loss =0.000924631
Time for epoch 3 is 0.2664499282836914 sec
gen_loss =0.9116014
disc_loss =0.00023492947
Time for epoch 4 is 0.27224302291870117 sec
gen_loss =0.9056747
disc_loss =0.00026061886
Time for epoch 5 is 0.2669079303741455 sec
gen_loss =0.8990334
disc_loss =5.815428e-06
Time for epoch 6 is 0.2687861919403076 sec
gen_loss =0.8927488
disc_loss =3.5935485e-05
Time for epoch 7 is 0.2731499671936035 sec
gen_loss =0.8870992
disc_loss =0.0003796812
Time for epoch 8 is 0.2754180431365967 sec
gen_loss =0.88220674
disc_loss =4.0746654e-05
Time for epoch 9 is 0.27419018745422363 sec
gen_loss =0.87791157
disc_loss =6.372533e-05
Time for epoch 10 is 0.2753419876098633 sec
gen_loss =0.874303
disc_loss =-2.339689e-06
Time for epoch 11 is 0.2717399597167969 sec
gen_loss =0.8710147
disc_loss =7.4

gen_loss =0.8341687
disc_loss =0.0
Time for epoch 100 is 0.27437424659729004 sec
gen_loss =0.83409244
disc_loss =0.0
Time for epoch 101 is 0.27285075187683105 sec
gen_loss =0.83402616
disc_loss =2.8393379e-06
Time for epoch 102 is 0.2710540294647217 sec
gen_loss =0.8339731
disc_loss =0.0
Time for epoch 103 is 0.27596330642700195 sec
gen_loss =0.8339141
disc_loss =-8.040312e-06
Time for epoch 104 is 0.2733759880065918 sec
gen_loss =0.8338752
disc_loss =0.0
Time for epoch 105 is 0.35964107513427734 sec
gen_loss =0.8338212
disc_loss =0.0
Time for epoch 106 is 0.2688431739807129 sec
gen_loss =0.8337707
disc_loss =0.0
Time for epoch 107 is 0.2684929370880127 sec
gen_loss =0.83370495
disc_loss =0.0
Time for epoch 108 is 0.2731893062591553 sec
gen_loss =0.83365875
disc_loss =0.0
Time for epoch 109 is 0.27278685569763184 sec
gen_loss =0.8336231
disc_loss =3.25379e-07
Time for epoch 110 is 0.27284812927246094 sec
gen_loss =0.83360326
disc_loss =0.0
Time for epoch 111 is 0.2693898677825928 sec
g

gen_loss =0.83274454
disc_loss =0.0
Time for epoch 200 is 0.2662651538848877 sec
gen_loss =0.83276886
disc_loss =0.0
Time for epoch 201 is 0.26146793365478516 sec
gen_loss =0.8327984
disc_loss =0.0
Time for epoch 202 is 0.26451683044433594 sec
gen_loss =0.83277255
disc_loss =-6.347301e-05
Time for epoch 203 is 0.260113000869751 sec
gen_loss =0.83294976
disc_loss =0.0
Time for epoch 204 is 0.24982404708862305 sec
gen_loss =0.8327741
disc_loss =0.0
Time for epoch 205 is 0.25548696517944336 sec
gen_loss =0.8327577
disc_loss =0.0
Time for epoch 206 is 0.25397825241088867 sec
gen_loss =0.83270633
disc_loss =0.0
Time for epoch 207 is 0.2617068290710449 sec
gen_loss =0.83275664
disc_loss =-0.0004398245
Time for epoch 208 is 0.25489115715026855 sec
gen_loss =0.832737
disc_loss =0.0
Time for epoch 209 is 0.26039814949035645 sec
gen_loss =0.8327151
disc_loss =0.0
Time for epoch 210 is 0.3264298439025879 sec
gen_loss =0.8327601
disc_loss =-5.6335888e-05
Time for epoch 211 is 0.2689540386199951 se

gen_loss =0.82437223
disc_loss =0.0
Time for epoch 299 is 0.25862574577331543 sec
gen_loss =0.82436216
disc_loss =0.0
Time for epoch 300 is 0.3421649932861328 sec
gen_loss =0.82435364
disc_loss =0.0
Time for epoch 301 is 0.2570161819458008 sec
gen_loss =0.8243475
disc_loss =0.0
Time for epoch 302 is 0.25510287284851074 sec
gen_loss =0.8243423
disc_loss =0.0
Time for epoch 303 is 0.2525489330291748 sec
gen_loss =0.8243388
disc_loss =0.0
Time for epoch 304 is 0.2518758773803711 sec
gen_loss =0.8243357
disc_loss =0.0
Time for epoch 305 is 0.2546360492706299 sec
gen_loss =0.8243337
disc_loss =0.0
Time for epoch 306 is 0.2510249614715576 sec
gen_loss =0.82433206
disc_loss =0.0
Time for epoch 307 is 0.25017523765563965 sec
gen_loss =0.8243307
disc_loss =0.0
Time for epoch 308 is 0.2505338191986084 sec
gen_loss =0.8243296
disc_loss =0.0
Time for epoch 309 is 0.2527167797088623 sec
gen_loss =0.8243288
disc_loss =0.0
Time for epoch 310 is 0.2491898536682129 sec
gen_loss =0.8243281
disc_loss =0.

gen_loss =0.8243316
disc_loss =0.0
Time for epoch 401 is 0.2577080726623535 sec
gen_loss =0.82432705
disc_loss =0.0
Time for epoch 402 is 0.2472541332244873 sec
gen_loss =0.8243251
disc_loss =0.0
Time for epoch 403 is 0.24953413009643555 sec
gen_loss =0.8243246
disc_loss =0.0
Time for epoch 404 is 0.24890518188476562 sec
gen_loss =0.8243244
disc_loss =0.0
Time for epoch 405 is 0.33100414276123047 sec
gen_loss =0.82432437
disc_loss =0.0
Time for epoch 406 is 0.2604079246520996 sec
gen_loss =0.82432437
disc_loss =0.0
Time for epoch 407 is 0.2503321170806885 sec
gen_loss =0.8243243
disc_loss =0.0
Time for epoch 408 is 0.24796485900878906 sec
gen_loss =0.8243243
disc_loss =0.0
Time for epoch 409 is 0.24790287017822266 sec
gen_loss =0.8243243
disc_loss =0.0
Time for epoch 410 is 0.2503511905670166 sec
gen_loss =0.8243243
disc_loss =0.0
Time for epoch 411 is 0.2522919178009033 sec
gen_loss =0.8243243
disc_loss =0.0
Time for epoch 412 is 0.248396635055542 sec
gen_loss =0.8243243
disc_loss =0.

In [23]:
final.eval

<bound method _EagerTensorBase.eval of <tf.Tensor: shape=(50, 151), dtype=float32, numpy=
array([[0.03341167, 0.        , 0.01789564, ..., 0.26842073, 0.        ,
        0.        ],
       [0.05432669, 0.24822022, 0.2433215 , ..., 0.10060824, 0.        ,
        0.08865923],
       [0.        , 0.        , 0.19622555, ..., 0.09512464, 0.        ,
        0.2490235 ],
       ...,
       [0.33203784, 0.00796748, 0.7549994 , ..., 0.33105403, 0.        ,
        0.2536575 ],
       [0.15126541, 0.02850102, 0.        , ..., 0.24078047, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.21970195, 0.19491848,
        0.1457395 ]], dtype=float32)>>

In [29]:
col = np.load('columns.npy', allow_pickle=True)

In [30]:
col.shape

(151,)

In [31]:
generated = pd.DataFrame(final, columns=col)

In [32]:
save = generated.to_pickle("klgen_data.pkl")


In [33]:

plt.figure(figsize=(10,8))
plt.plot(list(range(0,EPOCHS)),gloss)
plt.plot(list(range(0,EPOCHS)),dloss)
plt.title('Model Loss over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['generator loss','discriminator loss'])
plt.savefig('Learning_curve GAN least squares')
plt.show
plt.close()